In [ ]:
#

In [ ]:
#

In [ ]:
! pip install jiwer

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv("/kaggle/input/ipa-dataverse-added/output_dataverse.csv")
train_df = train_df[["Contents", "IPA"]]

train_df, val_df = train_test_split(train_df, test_size=0.1, shuffle=True, random_state=3000)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_pandas(train_df)
ds_eval = Dataset.from_pandas(val_df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_id = "google/byt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
def prepare_dataset(sample):
    output = tokenizer(sample["Contents"])
    output["labels"] = tokenizer(sample["IPA"])['input_ids']
    output["length"] = len(output["labels"])
    return output


ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
ds_eval = ds_eval.map(prepare_dataset, remove_columns=ds_eval.column_names)

In [ ]:
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": result}

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    group_by_length=True,
    length_column_name="length",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    metric_for_best_model="wer",
    greater_is_better=False,
    load_best_model_at_end=True,
    num_train_epochs=1,
    save_steps=13000,
    eval_steps=13000,
    logging_steps=13000,
    learning_rate=1e-4,
    weight_decay=5e-3,
    warmup_steps=500,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=512,
    push_to_hub=False,
    report_to="none",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.save_model(model_id)

In [ ]:
import pandas as pd
from transformers import pipeline

In [ ]:
test_df = pd.read_csv("/kaggle/input/regipa/test_regipa.csv")

In [ ]:
pipe = pipeline("text2text-generation", model=model_id, device=0)

In [ ]:
# preprocess-1: sort by length
index = test_df["Contents"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [ ]:
# preprocess-2: removing english alphabets
alpha_pat = "[a-zA-Z]"
test_df["Contents"] = test_df["Contents"].str.replace(alpha_pat, "", regex=True)

In [ ]:
# preprocess-3: adding period at the end of those texts not having period or '।'
indices_to_add_period = test_df[~test_df['Contents'].str.endswith('।')].index
test_df.loc[indices_to_add_period, 'Contents'] += '।'

In [ ]:
%%time
texts = test_df["Contents"].tolist()
ipas = pipe(texts, max_length=512, batch_size=64)
ipas = [ipa["generated_text"] for ipa in ipas]

In [ ]:
# postprocess-1
test_df["IPA"] = ipas
test_df = test_df.sort_index()

In [ ]:
# postprocess-2: removing period from the end of those ipas in which texts period were added in preprocess-3
test_df.loc[indices_to_add_period, 'IPA'] = test_df.loc[indices_to_add_period, 'IPA'].str.rstrip('।')
test_df.head()

In [ ]:
test_df.to_csv("submission2.csv", columns=["IPA"])

In [ ]:
df = pd.read_csv('/kaggle/working/submission2.csv')


# Check if there's any unnamed column
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]

if unnamed_columns:
    # Rename the first unnamed column to a desired name
    new_name = 'id'  # Specify your desired name here
    old_name = unnamed_columns[0]
    df.rename(columns={old_name: new_name}, inplace=True)


# Display the DataFrame after renaming the columns
print("\nDataFrame after renaming columns:")
df.rename(columns={'IPA': 'string'}, inplace=True)
print(df)


In [ ]:
df.isnull().sum()

In [ ]:
null_rows = df[df.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

In [ ]:
df_filled = df.fillna('missing')
print(df_filled)

In [ ]:
df_filled.to_csv('submission.csv', index=False)